In [1]:
import bqapi as bq
import pandas as pd
import numpy as np

sess = bq.Session()

In [2]:
hist_data = sess.get_historical_data(['spy us equity', 'vix index'], 'px_last', start_date='01-01-2010', 
                                     end_date='10-01-2015')
vix_data = hist_data.loc['vix index']
spy_data = hist_data.loc['spy us equity']

vol_data = sess.get_historical_data('spy us equity', 'px_volume', start_date='01-01-2010',
                                   end_date='10-01-2015')
spy_vol = vol_data.loc['spy us equity']

C:\blp\BQuant\environments\bqnt-0.15\lib\site-packages\bqnb\kernelapp.py:2: UserWarning: Function get_historical_data has been moved to pyrefdata and will be removed soon from bqapi. Please port your code to use BQL or pyrefdata.
  # All Rights Reserved.
C:\blp\BQuant\environments\bqnt-0.15\lib\site-packages\bqnb\kernelapp.py:7: UserWarning: Function get_historical_data has been moved to pyrefdata and will be removed soon from bqapi. Please port your code to use BQL or pyrefdata.
  


In [3]:
spy_returns = spy_data.pct_change()[1:]
vix_changes = vix_data.diff()[1:]
agg_df = pd.concat([spy_returns, vix_changes, spy_vol], axis=1).dropna(axis=0)
agg_df.columns = ['spy returns', 'vix changes', 'spy vol']

In [4]:
from bqplot import pyplot as plt
_axes_opts = {'x': {'grid_lines': 'solid', 'label': 'SPY Returns'}, 'y': {'grid_lines': 'solid', 'label': 'VIX Changes'}}

In [5]:
plt.figure()
scat = plt.scatter(agg_df['spy returns'], agg_df['vix changes'], default_colors=['mediumseagreen'], 
                   stroke='black', axes_options=_axes_opts, unselected_style={'opacity': 0.1})
plt.show(display_toolbar=False)

In [6]:
from ipywidgets import FloatRangeSlider
from IPython.display import display

In [7]:
vol_slid = FloatRangeSlider(min=0., max=1., step=0.01, value=[0., 1.], description='Volume Quantile')
display(vol_slid)

In [8]:
volume_sorted = np.sort(vol_data.values.flatten())
num_vols = len(volume_sorted) - 1

In [9]:
min_idx, max_idx = int(vol_slid.value[0] * num_vols), int(vol_slid.value[1] * num_vols)

In [10]:
print(min_idx, max_idx)

404 1446


In [11]:
volumes = vol_data.values.flatten()
sel_df = np.flatnonzero(np.logical_and((volumes > volume_sorted[min_idx]), (volumes < volume_sorted[max_idx])))

In [12]:
scat.selected = sel_df.tolist()